# **Simple Data Analysis Process**

## Context
This is a sample superstore dataset, a kind of a simulation where you can perform extensive data analysis to deliver insights on how the company can increase its profits while minimizing the losses. 

* **Dataset:** `sample-superstore.xlsx`
* **Download Dataset:**
    - https://www.kaggle.com/datasets/bravehart101/sample-supermarket-dataset
    - https://community.tableau.com/s/question/0D54T00000CWeX8SAL/sample-superstore-sales-excelxls
* **SQL Function:**
    - Join
    - Subquery
    - Min/Max/Sum/Group By

In [15]:
# !pip install Flask-SQLAlchemy
# !pip install flask_bcrypt
# !pip install Flask-Login

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from flask_login import LoginManager

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import datetime

In [2]:
# !pip install pandasql
import pandasql

**Data Collection**

In [3]:
orders = pd.read_excel('sample-superstore-orders.xlsx')
customer = pd.read_excel('sample-superstore-customer.xlsx')
# customer = pd.read_excel('Sample-Superstore.xlsx', sheet_name='Customer')

In [4]:
display(customer.head(1)) # 9.994 rows × 5 columns
display(orders.head(1)) # 9.994 rows × 18 columns

,CustomerID,CustomerName,Segment,City,State
0,12520,Claire Gute,Consumer,Henderson,Kentucky


,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,Country/Region,PostalCode,Region,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit,Unnamed: 17
0,1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,12520,United States,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,0.0


**Exploratory data analysis**

*1. Count the number of customer per city*

In [5]:
query = """
        select
            City, count(distinct CustomerID) as total_user
        from 
            customer
        group by City
        order by total_user desc; 
        """
df_city = pandasql.sqldf(query)
df_city

,City,total_user
0,New York City,355
1,Los Angeles,304
2,Philadelphia,239
3,San Francisco,232
4,Seattle,188
...,...,...
526,Antioch,1
527,Andover,1
528,Altoona,1
529,Abilene,1


*2. Count the number of orders per city*

In [6]:
query = """
        select
            t2.City, count(t1.OrderID)
        from 
            orders t1
        left join(select distinct CustomerID, City 
                  from customer)t2
        on t1.CustomerID = t2.CustomerID
        group by City
        order by count(t1.OrderID) desc; 
        """
df_obc = pandasql.sqldf(query)
df_obc

,City,count(t1.OrderID)
0,New York City,5024
1,Los Angeles,4393
2,Philadelphia,3452
3,San Francisco,3236
4,Seattle,2662
...,...,...
526,Atlantic City,4
527,Arlington Heights,4
528,Richardson,3
529,Ormond Beach,2


*3. Find first order date of each customer*

In [7]:
query = """
        select
            CustomerID, min(OrderDate) as first_buy_date
        from 
            orders as orders
        group by CustomerID
        order by first_buy_date; 
        """
df_odc = pandasql.sqldf(query)
df_odc

,CustomerID,first_buy_date
0,13000,2017-01-03 00:00:00.000000
1,19195,2017-01-04 00:00:00.000000
2,18085,2017-01-05 00:00:00.000000
3,15145,2017-01-06 00:00:00.000000
4,17230,2017-01-06 00:00:00.000000
...,...,...
788,15385,2020-07-08 00:00:00.000000
789,21145,2020-09-15 00:00:00.000000
790,18790,2020-10-21 00:00:00.000000
791,15700,2020-10-23 00:00:00.000000


*4. Find the number of customer who made their first order in each city, each day*

In [8]:
# query = """
#         select
#             t2.City, t1.first_order_date, count(CustomerID) number_of_users
#         from(
#             select CustomerID, min(OrderDate) first_order_date
#             from orders
#             group by 1
#             )t1
#         left join customer t2 on t1.CustomerID = t2.CustomerID
#         group by City; 
#         """
# df_cfo = pandasql.sqldf(query)
# df_cfo

*5. Find the first order GMV (Sales) of each customer. If there is a tie, use the order with the lower OrderID*

In [9]:
# query = """
#         select
#             t1.CustomerID, t1.sales
#         from orders t1
#         inner join(
#             select CustomerID, min(OrderDate) first_order_date
#             from(select CustomerID, OrderDate
#             from orders
#             order by OrderID
#             )
#         group by 1
#         )t2
#         on t1.CustomerID = t2.CustomerID
#         and t2.first_order_date = t2.order_date
#         ; 
#         """
# df_gmv = pandasql.sqldf(query)
# df_gmv